In [ ]:
%matplotlib widget
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import cm
import open3d as o3d

from planeslam.general import plot_3D_setup
from utils import dist_filter, spherical_project

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the points
datapath = os.path.join(os.getcwd(), '..', 'data', '5_15_2022', 'fr_config_3')
P = np.load(os.path.join(datapath, 'pc_0.npy'))

In [ ]:
# Plot
ax = plot_3D_setup(P)
ax.scatter3D(P[:,0], P[:,1], P[:,2], s=1, marker='.') 

In [ ]:
# Distance filter
P_filter = dist_filter(P, 6)

In [ ]:
# Plot
ax = plot_3D_setup(P_filter)
ax.scatter3D(P_filter[:,0], P_filter[:,1], P_filter[:,2], s=1, marker='.') 

In [ ]:
# Ground removal
lidar_height = 0.15
P_filter = P_filter[P_filter[:,2] > -lidar_height,:] 

In [ ]:
# Plot
ax = plot_3D_setup(P_filter)
ax.scatter3D(P_filter[:,0], P_filter[:,1], P_filter[:,2], s=1, marker='.') 

In [ ]:
# Background (Wall) Removal


In [ ]:
# DBSCAN clustering
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(P_filter)

In [ ]:
import time

start_time = time.time()
labels = np.array(pcd.cluster_dbscan(eps=0.2, min_points=10, print_progress=True))
print(time.time() - start_time)
clusters = [[] for _ in range(labels.max()+1)]
for i in range(len(labels)):
    if labels[i] >= 0:
        clusters[labels[i]].append(i)

In [ ]:
# Plot
ax = plot_3D_setup(P_filter)

color_idx = 0

for i, c in enumerate(clusters):
    cluster_pts = P_filter[c,:]
    ax.scatter3D(cluster_pts[:,0], cluster_pts[:,1], cluster_pts[:,2], s=1, color='C'+str(color_idx), marker='.')  
    color_idx += 1

In [ ]:
# Cluster centroids
cluster_centroids = [np.mean(P_filter[c], axis=0) for c in clusters]
cluster_centroids = np.asarray(cluster_centroids)

# Sort by distance
cluster_dists = np.linalg.norm(cluster_centroids, axis=1)

In [ ]:
num_landmarks = 4
landmark_cluster_idxs = np.argsort(cluster_dists)[:num_landmarks]
landmark_clusters = [clusters[i] for i in landmark_cluster_idxs]

In [ ]:
# Plot
ax = plot_3D_setup(P_filter)

color_idx = 0

for i, c in enumerate(landmark_clusters):
    cluster_pts = P_filter[c,:]
    ax.scatter3D(cluster_pts[:,0], cluster_pts[:,1], cluster_pts[:,2], color='C'+str(color_idx), marker='.')  
    color_idx += 1

In [ ]:
thetas, phis = spherical_project(P_filter)

In [ ]:
# Color code by distance
dists = np.linalg.norm(P_filter, axis=1)
min_dist = min(dists)
max_dist = max(dists)

In [ ]:
cmap = cm.get_cmap('plasma')
c_idx = list(np.interp(dists, [min_dist, max_dist], [255, 0]).astype(int))
colors = [cmap.colors[i] for i in c_idx]

fig, ax = plt.subplots()
c_idx = np.interp(dists, [min_dist, max_dist], [0, 255]).astype(int)
ax.scatter(thetas, phis, color=colors, s=1, marker='.')

In [ ]:
# Ground-plane removal?
# Background removal?